In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np 
import anndata as an
import os
import pickle

**Parameters**

In [2]:
root_folder =  "/scratch/devel/saguilar/PhD/PROJECTS/tonsil_atlas/TonsilAtlasCAP/"
data_folder = root_folder + "data/"

# Load data

In [3]:
# Load the h5ad file
adata = sc.read_h5ad(data_folder + "TonsilAtlasADATA.h5ad")

In [4]:
adata

AnnData object with n_obs × n_vars = 377988 × 38606
    obs: 'barcode', 'donor_id', 'gem_id', 'library_name', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cohort_type', 'cause_for_tonsillectomy', 'is_hashed', 'preservation', 'nCount_RNA', 'nFeature_RNA', 'pct_mt', 'pct_ribosomal', 'pDNN_hashing', 'pDNN_scrublet', 'pDNN_union', 'scrublet_doublet_scores', 'S.Score', 'G2M.Score', 'Phase', 'scrublet_predicted_doublet', 'doublet_score_scDblFinder', 'annotation_level_1', 'annotation_level_1_probability', 'annotation_figure_1', 'annotation_20220215', 'annotation_20220619', 'annotation_20230508', 'annotation_20230508_probability', 'UMAP_1_level_1', 'UMAP_2_level_1', 'UMAP_1_20220215', 'UMAP_2_20220215', 'UMAP_1_20230508', 'UMAP_2_20230508', 'type', 'azimuth.celltype.l1', 'azimuth.celltype.l2', 'tissue', 'organism', 'disease'
    var: 'ensembl_id', 'gene_symbol'
    uns: 'ArrayExpress', 'Azimuth', 'GitHub', 'HCATonsilData', 'Study', 'X_name', 'Zenodo'
    obsm: 'X_harmony', 'X_pca', 'X_umap

# Process data

## OBS

### Clarify in-house metadata

In [9]:
# GemID
adata.obs.rename(columns={'gem_id': '10X_channel'}, inplace=True)
# LibraryID
adata.obs.rename(columns={'library_name': 'libraryID'}, inplace=True)
# Annotation level 1
adata.obs.rename(columns={'annotation_level_1': 'cell_types_level1'}, inplace=True)
# Annotation level 2
adata.obs.rename(columns={'azimuth.celltype.l2': 'cell_types_level2'}, inplace=True)
# Annotation level 3
#adata.obs.rename(columns={'annotation_20220619': 'cell_types_level3_20220619'}, inplace=True)
# Annotation level 3
adata.obs.rename(columns={'annotation_20230508': 'cell_types_level3_20230508'}, inplace=True)

In [11]:
# We have a FDC cluster annotated as unknown that is NA for the azimuth annotation
na_mask = adata.obs["cell_types_level2"].isna()
adata.obs["cell_types_level2"] = adata.obs["cell_types_level2"].cat.add_categories(["unknown"])
adata.obs.loc[na_mask, "cell_types_level2"] = "unknown"

### Assay

In [13]:
listGroups = adata.obs['assay'].unique()
listGroups

['10x 3' v3']
Categories (1, object): ['10x 3' v3']

In [14]:
adata.obs['assay'] = adata.obs['assay'].replace({'3P': '10x 3\' v3.1', 'multiome': '10X multiome 5\' v1.1'})

### Disease

In [15]:
adata.obs['disease'] = "healthy"

### Organism

In [16]:
adata.obs['organism'] = "Homo sapiens"

### Tissue

In [17]:
adata.obs['tissue'] = "Tonsil"

### Subset to interesting columns

In [18]:
adata

AnnData object with n_obs × n_vars = 377988 × 38606
    obs: 'barcode', 'donor_id', '10X_channel', 'libraryID', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cohort_type', 'cause_for_tonsillectomy', 'is_hashed', 'preservation', 'nCount_RNA', 'nFeature_RNA', 'pct_mt', 'pct_ribosomal', 'pDNN_hashing', 'pDNN_scrublet', 'pDNN_union', 'scrublet_doublet_scores', 'S.Score', 'G2M.Score', 'Phase', 'scrublet_predicted_doublet', 'doublet_score_scDblFinder', 'cell_types_level1', 'annotation_level_1_probability', 'annotation_figure_1', 'annotation_20220215', 'annotation_20220619', 'cell_types_level3_20230508', 'annotation_20230508_probability', 'UMAP_1_level_1', 'UMAP_2_level_1', 'UMAP_1_20220215', 'UMAP_2_20220215', 'UMAP_1_20230508', 'UMAP_2_20230508', 'type', 'azimuth.celltype.l1', 'cell_types_level2', 'tissue', 'organism', 'disease'
    var: 'ensembl_id', 'gene_symbol'
    uns: 'ArrayExpress', 'Azimuth', 'GitHub', 'HCATonsilData', 'Study', 'X_name', 'Zenodo'
    obsm: 'X_harmony', 'X_pca', '

In [19]:
columns_to_keep = ['barcode', 'donor_id', '10X_channel', 'libraryID', 'assay', 'sex', 'age', 'age_group', 'hospital',
                  'cell_types_level1', 'cell_types_level2', 'cell_types_level3_20230508', 'disease', 'organism', 'tissue']
adata.obs = adata.obs[columns_to_keep]
adata

AnnData object with n_obs × n_vars = 377988 × 38606
    obs: 'barcode', 'donor_id', '10X_channel', 'libraryID', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cell_types_level1', 'cell_types_level2', 'cell_types_level3_20230508', 'disease', 'organism', 'tissue'
    var: 'ensembl_id', 'gene_symbol'
    uns: 'ArrayExpress', 'Azimuth', 'GitHub', 'HCATonsilData', 'Study', 'X_name', 'Zenodo'
    obsm: 'X_harmony', 'X_pca', 'X_umap'

## UNS

In [20]:
adata.uns['DOI'] = adata.uns.pop('Study')
adata.uns['X_layer_explanation'] = "raw counts from filtered cells (utilized in the paper) with ENSG ids"
del adata.uns['X_name']

In [21]:
adata

AnnData object with n_obs × n_vars = 377988 × 38606
    obs: 'barcode', 'donor_id', '10X_channel', 'libraryID', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cell_types_level1', 'cell_types_level2', 'cell_types_level3_20230508', 'disease', 'organism', 'tissue'
    var: 'ensembl_id', 'gene_symbol'
    uns: 'ArrayExpress', 'Azimuth', 'GitHub', 'HCATonsilData', 'Zenodo', 'DOI', 'X_layer_explanation'
    obsm: 'X_harmony', 'X_pca', 'X_umap'

## VAR

In [22]:
adata.var.head

<bound method NDFrame.head of                       ensembl_id      gene_symbol
ENSG00000290825  ENSG00000290825          DDX11L2
ENSG00000243485  ENSG00000243485      MIR1302-2HG
ENSG00000237613  ENSG00000237613          FAM138A
ENSG00000290826  ENSG00000290826  ENSG00000290826
ENSG00000186092  ENSG00000186092            OR4F5
...                          ...              ...
ENSG00000277836  ENSG00000277836  ENSG00000277836
ENSG00000278633  ENSG00000278633  ENSG00000278633
ENSG00000276017  ENSG00000276017  ENSG00000276017
ENSG00000278817  ENSG00000278817  ENSG00000278817
ENSG00000277196  ENSG00000277196  ENSG00000277196

[38606 rows x 2 columns]>

# Save adata

In [23]:
adata

AnnData object with n_obs × n_vars = 377988 × 38606
    obs: 'barcode', 'donor_id', '10X_channel', 'libraryID', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cell_types_level1', 'cell_types_level2', 'cell_types_level3_20230508', 'disease', 'organism', 'tissue'
    var: 'ensembl_id', 'gene_symbol'
    uns: 'ArrayExpress', 'Azimuth', 'GitHub', 'HCATonsilData', 'Zenodo', 'DOI', 'X_layer_explanation'
    obsm: 'X_harmony', 'X_pca', 'X_umap'

In [ ]:
save_path = root_folder + "results/TonsilAtlasADATA_toCAP.h5ad"
adata.write(save_path, compression="gzip")

In [ ]:
save_path = root_folder + "results/TonsilAtlasADATA_toCAP.h5ad"
